##### Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

In [44]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [4]:
#cargar el df unido
df = pd.read_csv("Customer_Flight_Loyalty_unido.csv", index_col=0) 
df.head(2)

,Loyalty Numberflight,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,100018,2017,1,3,0,3,1521,152.0,0,0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,73455.0,Divorced,Star,3839.61,Standard,2016,3,0,0


#Objetivo del Ejercicio: Utilizando un conjunto de datos que hemos compartido,se busca evaluar si existen diferencias
 significativas en el número de vuelos reservados según el nivel educativo de los clientes. 

#Seleccionar una Métrica Principal: promedio, la desviación estandar, los percentiles

#Seleccionar un Grupo de Control y un Grupo de Prueba:


In [39]:


#filtro por las columnas que necesito
df_filtrado = df[['Flights Booked', 'Education']]


# Crear una nueva columna 'Education Group' basada en 'Education'
df_filtrado['Education Group'] = df_filtrado['Education']

# Reemplazar los valores en 'Education Group' según la categoría

df_filtrado.loc[df_filtrado['Education'].isin(['Bachelor', 'Master', 'Doctor']), 'Education Group'] = 'Educación Superior'
df_filtrado.loc[df_filtrado['Education'].isin(['High School or Below', 'College']), 'Education Group'] = 'Educación No Superior'

#filtro por las columnas que necesito
df_filtrado = df_filtrado[['Flights Booked', 'Education Group']]

# Filtrar los datos por las nuevas categorías
df_superior = df_filtrado[df_filtrado['Education Group'] == 'Educación Superior']
df_no_superior = df_filtrado[df_filtrado['Education Group'] == 'Educación No Superior']

# Mostrar las primeras filas de cada DataFrame
print("Educación Superior:")
display(df_superior.head())

print("\nEducación No Superior:")
display(df_no_superior.head())


Educación Superior:


,Flights Booked,Education Group
0,3,Educación Superior
4,0,Educación Superior
5,0,Educación Superior
7,0,Educación Superior
8,6,Educación Superior



Educación No Superior:


,Flights Booked,Education Group
1,10,Educación No Superior
2,6,Educación No Superior
3,0,Educación No Superior
6,0,Educación No Superior
13,0,Educación No Superior


#Analizar los Datos

In [42]:


# Nombre de columnas
print("Columnas:")
display(df_filtrado.columns)
print("----"*10)

# Primeras filas
print("Primeras filas:")
display(df_filtrado.head(2))
print("----"*10)

# Información general
print("Información general:")
display(df_filtrado.info())
print("----"*10)

# Estadísticas descriptivas
print("Estadísticas descriptivas:")
display(df_filtrado.describe().T)
print("----"*10)

#Estadisticas descritivas por agrupacion de educacion 
df_grupos = df_filtrado.groupby("Education Group")["Flights Booked"]
display(df_grupos.describe().T)
print("----"*10)

# Verificación de valores nulos
print("Valores nulos por columna:")
display(df_filtrado.isnull().sum())
print("----"*10)

# Verificación de valores duplicados
print("Número de filas duplicadas:")
display(df_filtrado.duplicated().sum())
print("----"*10)

# Conteo de valores únicos
print("Número de valores únicos por columna:")
display(df_filtrado.nunique())
print("----"*10)

# Diferenciación entre variables categóricas y numéricas
print("Variables categóricas:")
display(df_filtrado.select_dtypes(include='object').columns)
print("----"*10)
print("Variables numéricas:")
display(df_filtrado.select_dtypes(include=['int64', 'float64']).columns)
print("----"*10)

# Dimensiones del DataFrame
print("Shape del DataFrame:")
display(df_filtrado.shape)


Columnas:


Index(['Flights Booked', 'Education Group'], dtype='object')

----------------------------------------
Primeras filas:


,Flights Booked,Education Group
0,3,Educación Superior
1,10,Educación No Superior


----------------------------------------
Información general:
<class 'pandas.core.frame.DataFrame'>
Index: 16618 entries, 0 to 16736
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Flights Booked   16618 non-null  int64 
 1   Education Group  16618 non-null  object
dtypes: int64(1), object(1)
memory usage: 389.5+ KB


...

Index(['Education Group'], dtype='object')

----------------------------------------
Variables numéricas:


Index(['Flights Booked'], dtype='object')

----------------------------------------
Shape del DataFrame:


(16618, 2)

#Pruebas estadísticas

In [56]:
# Para comprobar si las distribuciones del número de vuelos reservados son normales 
# para cada grupo de educación,utilizaré la prueba de Shapiro
# NORMALIDAD: 
print("---------"*10)
print("NORMALIDAD")
p_value = shapiro(df_filtrado['Flights Booked']).pvalue

if p_value > 0.05:
    print("Distribución normal:" , p_value)
else:
    print("Distribución no normal:" , p_value)


#HOMOGENEIDAD DE VARIANZAS
print("---------"*10)
print("HOMOGENEIDAD DE VARIANZAS")

# lo primero que tenemos que hacer es crear tantos conjuntos de datos para cada una de las 
# categorías que tenemos, Educación No Superior y Educación  Superior

valores_evaluar = []
media = df_filtrado['Flights Booked'].mean()
df_filtrado['Media'] = media

for valor in df_filtrado["Education Group"].unique():
    valores_evaluar.append(df_filtrado[df_filtrado["Education Group"]== valor]['Media'])

statistic, p_value = stats.levene(*valores_evaluar)
if p_value > 0.05:
    print(f"Para la Media las varianzas son homogéneas entre grupos.")
else:
    print(f"Para la Media, las varianzas no son homogéneas entre grupos.")

# Como no se ajusta a una distribución normal tendremos que hacer la prueba de Mann-Whitney
print("---------"*10)
print("MANN-WHITNEY")
media = df_no_superior['Flights Booked'].mean()
df_no_superior['Media'] = media

media = df_superior['Flights Booked'].mean()
df_superior['Media'] = media

statistic, p_value = mannwhitneyu(df_no_superior["Media"], df_superior["Media"], alternative="two-sided")
# Imprimimos los resultados
print('Estadístico U de Mann-Whitney:',statistic)
print('Valor p:',p_value)
# Interpretamos los resultados
if p_value > 0.05:
    print('No hay diferencia significativa entre los grupos.')
else:
    print('Hay diferencia significativa entre los grupos.')



------------------------------------------------------------------------------------------
NORMALIDAD
Distribución no normal: 2.3804443742362194e-95
------------------------------------------------------------------------------------------
HOMOGENEIDAD DE VARIANZAS
Para la Media, las varianzas no son homogéneas entre grupos.
------------------------------------------------------------------------------------------
MANN-WHITNEY
Estadístico U de Mann-Whitney: 57990505.0
Valor p: 0.0
Hay diferencia significativa entre los grupos.


La prueba de Mann-Whitney indica que el nivel educativo influye significativamente en el número de vuelos reservados cuando el valor p es menor que 0.05. 